In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/archive"

/content/drive/MyDrive/archive


In [5]:
# Create a directory named model to save your model

output_models_dir = "/content/drive/MyDrive/archive/model3"

In [6]:
#%%capture
!pip uninstall torchaudio
!pip install torchaudio
!pip install datasets==1.4.1
!pip install transformers==4.4.0

#!pip install torchaudio==0.7
!pip install jiwer
!pip install pandas
#!pip install pandas==1.1.5

Found existing installation: torchaudio 0.10.0+cu111
Uninstalling torchaudio-0.10.0+cu111:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchaudio-0.10.0+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchaudio/*
Proceed (y/n)? y
  Successfully uninstalled torchaudio-0.10.0+cu111
     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:37tcmalloc: large alloc 1147494400 bytes == 0x563946c70000 @  0x7f943f0d5615 0x56390db743bc 0x56390dc5518a 0x56390db771cd 0x56390dc69b3d 0x56390dbeb458 0x56390dbe602f 0x56390db78aba 0x56390dbeb2c0 0x56390dbe602f 0x56390db78aba 0x56390dbe7cd4 0x56390dc6a986 0x56390dbe7350 0x56390dc6a986 0x56390dbe7350 0x56390dc6a986 0x56390dbe7350 0x56390db78f19 0x56390dbbca79 0x56390db77b32 0x56390dbeb1dd 0x56390dbe602f 0x56390db78aba 0x56390dbe7cd4 0x56390dbe602f 0x56390db78aba 0x56390dbe6eae 0x56390db789da 0x56390dbe7108 0x56390dbe602f
     |████████████████████████████████|

In [11]:
%cd "/content/drive/MyDrive/archive/data" 

/content/drive/MyDrive/archive/data


In [12]:
import pandas as pd

data = pd.read_csv('Train_4000.csv')
data['ID'] = 'clips/' + data['ID'] + '.mp3'
data.to_csv('train.csv',index=False)
data.head()

,ID,up_votes,down_votes,age,gender,transcription
0,clips/0042cdb4d4a015cddacc26d88faffdd714b7a272...,2,0,twenties,male,géenel ma
1,clips/00439f02fa8f0dade934bdb317199b71662e9396...,2,0,twenties,male,jox ma
2,clips/00449100b205f8244174936b580e4084968ab7bb...,2,0,twenties,male,seetal ma
3,clips/004bb970ff5217dc663cd9d69cf6dccc8b419706...,2,0,twenties,male,soppil ma
4,clips/004bc77cc17a10c3964932a6e5acaa7494023d8f...,2,0,twenties,male,ubbil ma


In [13]:
# Create training folds(5 folds)

from datasets import load_dataset, load_metric,concatenate_datasets



vals_ds = load_dataset('csv',data_files=['train.csv'], split=[f'train[{k}%:{k+20}%]' for k in range(0, 100, 20)])


voice_val = vals_ds[0].remove_columns(['up_votes', 'down_votes', 'age', 'gender'])


trains_ds = load_dataset('csv',data_files=['train.csv'], split=[f'train[:{k}%]+train[{k+20}%:]' for k in range(0, 100, 20)])

voice_train = trains_ds[0].remove_columns(['up_votes', 'down_votes', 'age', 'gender'])

Using custom data configuration default-eecf0d7d1f390c39


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-eecf0d7d1f390c39/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


Using custom data configuration default-eecf0d7d1f390c39
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-eecf0d7d1f390c39/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)


In [14]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [15]:
# Display the dataset

show_random_elements(voice_train, num_examples=20)

,ID,transcription
0,clips/4bd6648597248e73a3218d7337c4f528c67303de9de5d59358cf3a223bfe1295cf2499924fc9d825267130e166556d4d885010c52ce4cf2debc4067c4521c56c.mp3,Philipe maguiléne Senghor
1,clips/65e15f3e70c249b14b22bf0d8e3fbb5cd1397e8ae7c6cd7b46579e334867f65180cddbd817a41c32f86028dcb96f0351ba53df452dbe578153cfbda695884ea3.mp3,Pont Mariste
2,clips/5f097bb86f862d523bf26af0c8d65746ebba5d08a70e386d61951fcfd059f1eafd118d069d20a2eae502fbfddb6380d5d97a1a1f426fd2df2d73efce85b4d88f.mp3,Garage Keur Massar
3,clips/01149a055891fabc8ce23689a9da0a754c304b571b4fd05bfb0c5550602d5c19b7caf2eb50cd252026aaaa2dd07ea50dd10ce4a32fde4773dd50d2d58849b130.mp3,Complexe Xelcom Bache
4,clips/AUDIO-2021-07-25-22-25-30.mp3,dindil ma
5,clips/44cc41db659fbe8d39155ac92874fbf7cc6ecb9f704ea031f8bf53be0c3cb02585659a833120b5546c54225fa6b0a0cf2406c62bf8ed74eed45959f83d504728.mp3,dama bëgg dem Dakar Nave
6,clips/4646d9e2b74646466d2bd4c5d4b3861d9df911b10dadc20fb2dcbb0c053fc41bdfc1df3b75e3e010e57620ec4356217ee7f0cfb56d84cd04acebc537b79f5c37.mp3,Marché bou Béss
7,clips/46d0ea33d4cc2668fb298f2a5356887aa8ed0b66163c0c0174391db775ba862f5b4beb4918f671514d60e41604baed107f3347b7833847b1e2bcfc2a66205ba8.mp3,Tableau Ferrailes
8,clips/36db3eb42081c95bbfae0d6a7ac4ecd121c7e201750129599aa5cd3b1cb74149119d5c5846e6e7efbd0842ace66094ff00110df5ff3c0de30c83646844e288f5.mp3,Cinéma Thiaroye Gare
9,clips/1b310ad6b1287fb3ba08c067bfc528870d48a768329578ae9cdc293f0d1e4efbeee4dc5445b63c4f8692c3c75e8d9c84039cd07392a7b54c4e7aa39bec8421f8.mp3,Soumbédioune


In [16]:
import re
chars_to_ignore_regex = '[\,\؟\.\!\-\;\:\"\)\(\«\»\؛\—\ـ\_\،\“\%\‘\”]'

def remove_special_characters(batch):  
    batch["transcription"] = re.sub(chars_to_ignore_regex,' ',batch["transcription"]).lower()+" "
    return batch


voice_train = voice_train.map(remove_special_characters)
voice_val = voice_val.map(remove_special_characters) 

In [17]:
# Display cleaned data

show_random_elements(voice_train, num_examples=20)

,ID,transcription
0,clips/427468326972bfae0bd0f4befcb245faead05e3b9cfeee4642caebd441ebac1f6fd7fceef3608eaec10e3afa02ed29fd0c8c16ef360e4a3d7170c0fb2f8b5072.mp3,cité port
1,clips/1be1b4478c2ef142e9a9e91b0498699fc5bfc0135f24f0b78f8c892001123191d549e8f2cc7491c7d5c2fd8252ab2445c253c726c3859b713157b91e51054eec.mp3,gare feroviére de dakar
2,clips/5ddbacee745235e022211c0a0cd564c2d3b75b8074e3d3c50b58a4db3e1c75684b08b8b0f93ee920715f1077c75bc75ba6da4a295146a3a090fd470fba276e58.mp3,pharmacie sedami
3,clips/6747aac10a390b65965e5699bdc01cc7745f89bc6bd64795df9a5ce1f0691a29c84521a6d5f88c0e02f6c66efa039570a38b91b86ea9acf8fa72ccc00a0601c2.mp3,terrain niague
4,clips/64039482c79db13b62348955932b569547e4b4f748341aa66cd6e066727ccb93a29d261911151335694ad1a90bfbb253aaefbb588715e6ddc7e33c29b8885300.mp3,cité sonatel liberté
5,clips/6b87bd66017d560634a8ea978c0bfc36a98815124bf1bd801d556dcfafde537242b21edcced45eeecf81099915ab3194b2aa3b2ef13712b1bf1bc5caf0c8a231.mp3,croisement bountou pikine
6,clips/5e6af83e6628a99045a5a4ca468c145e2c9bbe7f7fa8e9591a02edef7629e4f5ef33047e9ad653422bef838caec784d1427d458b5f5511f3edd8274712059665.mp3,cité tacko kounoune
7,clips/5eb25bbac6bb6748351f4b6474657984e56cea0d468658bbd50c71e6da6bbc8b103c559282ae1d6ff0a5cc9a52ab972a85603e1039810b1256db1f24221048f1.mp3,just for u
8,clips/54a46a22a98579833470f36a71652c33ddb9d59ae82aef7943f0481776fcc17cd8aa1f35eb176c1ebe755f071ff7840885b22cb33615b9688f53173590382922.mp3,fass mbao
9,clips/3f604e28ac2554194bca50e95c9c1071556cb09a1b8a236f157c5c626e8fbe87fe75dbc7d30eeac60dfe438f4021a7afa0d22b0d623c42082c502ff5d8168fb1.mp3,marchée grand yoff


In [18]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [19]:
vocab_train = voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=voice_train.column_names)
vocab_val = voice_val.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=voice_val.column_names)

In [20]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_val["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

In [22]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [23]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

38

In [24]:
# Prepare vocabulary
import json
with open(f"{output_models_dir}/vocab.json", 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [28]:
# Tokenization
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(f"{output_models_dir}/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [25]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [29]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [30]:
processor.save_pretrained(output_models_dir)

!apt install git-lfs
repo_name = "https://huggingface.co/fkHug/model3FromWav2vec"

!git config --global user.email "kinendiaye4991@gmail.com"
!git clone https://huggingface.co/fkHug/model3FromWav2vec

%cd "model3FromWav2vec"
tokenizer.save_pretrained(repo_name)
processor.save_pretrained(repo_name)


!git add . && !git commit -m "update from $fkHug"
!git push


In [31]:
voice_train[0]

{'ID': 'clips/1836923a3ea41e969e14e4657bea776266b74ae5848c1a4b33e097d7fac541354b60debeb76f7cf1d6dfed17608fa278c08b71d63a59c5d50cd86634730dff8c.mp3',
 'transcription': 'dépot dakar dem dikk laa bëgg dem '}

In [32]:
import torchaudio
import torch

# Apply transformation to 0.5 of the training data(reduce speed + reverbration)

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["ID"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["transcription"]
    return batch

    #speech_array, sampling_rate = torchaudio.load(batch["path"])
    #batch["speech"] = speech_array[0].numpy()
    #batch["sampling_rate"] = sampling_rate
    #batch["target_text"] = batch["sentence"]
    #return batch
def speech_file_to_array_fn_aug(batch):
    speech_array, sampling_rate = torchaudio.load(batch["ID"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
    #batch["speech"] = resampler(speech_array).squeeze().numpy()
    effects = [
           ["lowpass", "-1", "300"], # apply single-pole lowpass filter
           ["speed", "0.9"],  # reduce the speed
                     # This only changes sample rate, so it is necessary to
                     # add `rate` effect with original sample rate after this.
           ["rate", f"{sampling_rate}"],
           #["reverb", "-w"],  # Reverbration gives some dramatic feeling
           ]
    effects2 = [["reverb","-w"]]

    # Apply effects
    prob = torch.rand(1)
    rever = torch.rand(1)
    if prob > 0.5:
      waveform2, sample_rate2 = torchaudio.sox_effects.apply_effects_tensor(speech_array, sampling_rate, effects) 
      batch["speech"] = resampler(waveform2).squeeze().numpy()
      batch["sampling_rate"] = 16000
      batch["target_text"] = batch["transcription"]
    if rever>0.5:
      waveform3, sample_rate3 = torchaudio.sox_effects.apply_effects_tensor(speech_array, sampling_rate, effects2) 
      batch["speech"] = resampler(waveform3).squeeze().numpy()
      batch["sampling_rate"] = 16000
      batch["target_text"] = batch["transcription"]  


    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["transcription"]
    return batch
#voice_val = voice_val.map(speech_file_to_array_fn, remove_columns=voice_val.column_names,num_proc=1)




In [33]:
voice_train = voice_train.map(speech_file_to_array_fn_aug,remove_columns=voice_train.column_names,num_proc=1) 
voice_val = voice_val.map(speech_file_to_array_fn, remove_columns=voice_val.column_names,num_proc=1)

In [ ]:
#tokenizer.save_pretrained("https://huggingface.co/fkHug/model3FromWav2vec")
#processor.save_pretrained("https://huggingface.co/fkHug/model3FromWav2vec")

In [34]:
import IPython.display as ipd
import numpy as np
import random
# Play audio
rand_int = random.randint(0, len(voice_train)-1)

ipd.Audio(data=np.asarray(voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [35]:
rand_int = random.randint(0, len(voice_train)-1)

print("Target text:", voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", voice_train[rand_int]["sampling_rate"])

Target text: orca 
Input array shape: (32640,)
Sampling rate: 16000


In [36]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [37]:
voice_train = voice_train.map(prepare_dataset, remove_columns=voice_train.column_names, batch_size=8, num_proc=4, batched=True)
voice_val = voice_val.map(prepare_dataset, remove_columns=voice_val.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [38]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [39]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [40]:
wer_metric = load_metric("wer")

In [41]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [42]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.05,
    hidden_dropout=0.05,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [44]:
model.freeze_feature_extractor()

In [46]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=output_models_dir,
  #output_dir="dev/",
  group_by_length=True,
  per_device_train_batch_size=16,
  #per_device_eval_batch_size=32,
  gradient_accumulation_steps=2,
  dataloader_num_workers = 1,
  evaluation_strategy="steps",
  num_train_epochs=10,
  #fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  #warmup_steps=1000,
  warmup_steps=500,
  save_total_limit=1
  #save_total_limit=2
)

In [47]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=voice_train,
    eval_dataset=voice_val,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train() 

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Step,Training Loss,Validation Loss


In [ ]:
%cd "/content/drive/MyDrive/archive/data" 

/content/drive/MyDrive/archive/data


In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Inference on test data

test_dataset = load_dataset('csv',data_files=['SampleSubmission.csv'])



processor = Wav2Vec2Processor.from_pretrained(output_models_dir)
model = Wav2Vec2ForCTC.from_pretrained(f"{output_models_dir}/checkpoint-1200")
model.to("cuda")


# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(f"clips/{batch['ID']}.mp3")
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch
  

test_dataset = test_dataset.map(speech_file_to_array_fn)

def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding=True)

    with torch.no_grad():
         logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["transcription"] = processor.batch_decode(pred_ids)
    return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("Prediction:", result)


Using custom data configuration default-e20ecb1ad4bd0bbc
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e20ecb1ad4bd0bbc/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1



Prediction: DatasetDict({
    train: Dataset({
        features: ['ID', 'speech', 'transcription'],
        num_rows: 234
    })
})


In [ ]:
import pandas as pd


sub = pd.read_csv('SampleSubmission.csv')
sub.head()

,ID,transcription
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,0
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,0
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,0
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,0
4,01337d7daaf50c9a53b783f4cd6704b5a152dd5d299899...,0


In [ ]:
result

DatasetDict({
    train: Dataset({
        features: ['ID', 'speech', 'transcription'],
        num_rows: 234
    })
})

In [ ]:
# fill the empty audio with None
final_pred = [ 'None' if pred=='' else pred for pred in result['train']['transcription']]
final_pred[200]
        

'gendarmerie national sicée desse eaur'

In [ ]:
sub['transcription'] =  final_pred
sub.to_csv('submission.csv',index=False)
sub.head(10)

,ID,transcription
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,africatel arvés
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,nan laay dem ngi dem tally bou bes
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,aficatel avéss
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,mosquée de cambéréne
4,01337d7daaf50c9a53b783f4cd6704b5a152dd5d299899...,ecole hlm rufisque
5,016a99acf910b17ae7e2780ae71884c71dea8eeb1d4a51...,rue baf faséne
6,01eb602b69c303a54315a272c3544daba1eb3a3341b1ec...,ecole primair privéelais petit génis
7,0235405604f73d5ba9500dc44c293e6ebcb7575f830c47...,marché ouakam
8,0266e6e8f354f4a241b744a64a226d24b90aeef8160ec8...,dieupeule diklé
9,027d12b2a3bd049c026d2103b849de47076a785f76c174...,barny rails


In [ ]:
import IPython.display as ipd
import numpy as np

speech_array, sampling_rate = torchaudio.load(f"clips/{sub['ID'][4]}.mp3")
resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
batch = resampler(speech_array).squeeze().numpy()
ipd.Audio(data=np.asarray(batch), autoplay=True, rate=16000)